In [1]:
# !pip install pytesseract

In [7]:
import os

In [29]:
def AIopinion(messages):
    from openai import OpenAI

    YOUR_API_KEY = ""
    
    import re
    from IPython.display import clear_output
    
    client = OpenAI(api_key=YOUR_API_KEY, base_url="https://api.perplexity.ai")
    
    try:
        response_stream = client.chat.completions.create(
            model="sonar-pro", #sonar-pro #sonar-reasoning-pro #sonar #sonar-reasoning
            messages=messages,
            stream=True,
        )
    
        full_response = []
        link_map = {}
        link_counter = 1
        buffer = ""
    
        for chunk in response_stream:
            if hasattr(chunk, 'citations') and chunk.citations:
                # Enumerate URLs starting at index 1 for proper citation numbering
                link_map = {str(i): url for i, url in enumerate(chunk.citations, start=1)}
            if chunk.choices and chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                buffer += content
                
                # Process URLs incrementally
                while True:
                    match = re.search(r'https?://[^\s\)\]\}]+', buffer)
                    if not match:
                        break
                        
                    url = match.group()
                    if url not in link_map:
                        link_map[url] = link_counter
                        link_counter += 1
                    
                    # Replace URL with reference marker
                    buffer = buffer.replace(url, f'[{link_map[url]}]', 1)
                
                # Print processed content and clear buffer
                print(buffer, end='', flush=True)
                full_response.append(buffer)
                buffer = ""
    
        # Print remaining buffer content
        if buffer:
            print(buffer, end='', flush=True)
            full_response.append(buffer)
    
        # Add references appendix
        print("\n\n--- References ---")
        for url, num in sorted(link_map.items(), key=lambda x: x[1]):
            print(f"[{num}] {url}")
    
    except Exception as e:
        clear_output()
        print(f"Error occurred: {str(e)}")

In [30]:
from PIL import Image
import pytesseract  # First install: pip install pytesseract AND tesseract-OCR

# Convert image to analyzable text
def image_to_analysis(image_path):
    image = Image.open(image_path)
    text_data = pytesseract.image_to_string(image)
    
    # Clean up OCR results
    cleaned_data = "\n".join([line for line in text_data.split('\n') if line.strip()])
    return cleaned_data

In [31]:
def persist_chart_for_analysis(url, image_path):
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options

    options = Options()
    options.add_argument("--headless")  # Run in background
    driver = webdriver.Chrome(options=options)

    driver.get(url)
    driver.implicitly_wait(15)  # Wait for chart to load

    # Take full-page screenshot
    driver.save_screenshot(image_path)
    driver.quit()
    

In [32]:
image_path = 'chart.png'
url = "https://www.tradingview.com/script/Edd4Wt4b-My-Momentum-Strategy/"
# url = "https://s.tradingview.com/widgetembed/?symbol=NASDAQ:AAPL&interval=1D&theme=dark&studies=RSI@tv-basicstudies,MACD@tv-basicstudies"

In [33]:
persist_chart_for_analysis(url=url, image_path=image_path)
processed_data = image_to_analysis(image_path)

In [34]:
print(processed_data)

17 Q Search
a Updated 4 hours ago
My Momentum Strategy
Strategy report Source code
® Bitcoin / U.S. Dollar - 30.1 AU STON 9.96 Hg H te -Size-{No-of Data Points in-Each-Cluster) Cu lity
= i
sell sel 3 (bigh} 60
2 (Medium)
4 98,000
96,876
=) —| 96,551
See BICUSD) 96,544
Close entry(s) ordér sell alee 96,522
ae +1 buybuybuy 96,261
+1-+1-+1 [ee
95,000
ee
—I7-TradingVview- (Se ——— —————————
- a
06:00 09:00 12:00 15:00 18:00 21:00 22 03:00 06:00 09:00


In [35]:
messages = [{
    "role": "user",
    "content": f"""Analyze this data:
    
    ## 
    {processed_data}
    """
}]

In [36]:
AIopinion(messages=messages)

Based on the image provided, it appears to be a chart showing a momentum trading strategy for Bitcoin (BTC/USD). Here's an analysis of the key elements:

1. Time frame: The chart shows 30-minute candles, covering approximately 24 hours of trading activity.

2. Price range: The Bitcoin price ranges from around $95,000 to $98,000 during the period shown.

3. Trading signals: The strategy generates buy and sell signals:
   - There are multiple "buy" signals indicated by green arrows pointing up, typically at local price lows.
   - There are "sell" signals indicated by red arrows pointing down, typically at local price highs.

4. Clustering: The chart shows three levels of clustering, likely based on price action or volume:
   - High (3): Represented by red dots
   - Medium (2): Represented by yellow dots
   - Low (1): Represented by green dots

5. Order management: There's an indication to "Close entry(s) order sell" at one point on the chart, suggesting the strategy includes rules for ex